# NeMo ASR Training Using AWS SageMaker

In this tutorial we show how you can train a NeMo ASR Model using [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html). This is meant to be a minimalistic example of how to use SageMaker with NeMo.

AWS SageMaker is useful for practioners/researchers who are familiar with training locally or on a remote instance (via SSH). SageMaker also supports multi-GPU & Multi-node.

Using AWS SageMaker we train a simple Conformer CTC model using the AN4 dataset on a remote instance with a GPU (p3.2xlarge). We use S3 to store the data and our checkpoints/logs.

The overall steps are:

1. Setup your AWS Credentials to access SageMaker
2. Download the source code we'll be running
3. Setup AN4 dataset, upload data to S3
4. Configure the training job
5. Run training job on SageMaker
6. Download model, (Optional) Tensorboard Logs

In [ ]:
install_needed = True  # should only be True once
# install_needed = False

In [ ]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U datasets transformers
    !{sys.executable} -m pip install -U wget omegaconf text-unidecode
    ## Install NeMo
    BRANCH = 'main'
    !{sys.executable} -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
    !pip install --upgrade --force-reinstall llvmlite
    IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
# """
# You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

# Instructions for setting up Colab are as follows:
# 1. Open a new Python 3 notebook.
# 2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
# 3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
# 4. Run this cell to set up dependencies.
# 5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
# """
# # If you're using Google Colab and not running locally, run this cell.

# ## Install dependencies
# !pip install wget
# !sudo yum install sox -y
# !sudo yum install libsndfile
# !sudo sh ./packages/ffmpeg/install-ffmpeg-amazon-linux.sh
# !pip install numba==0.53.1
# !pip install text-unidecode
# !pip install matplotlib>=3.3.2
# !pip install sagemaker==2.118.0 awscli sagemaker-training
# !pip install librosa==0.9.2

# BRANCH = 'main'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# """
# Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
# Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
# that you want to use the "Run All Cells" (or similar) option.
# """
# # exit()

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=e0043256daa791487223cd187586dd9cfe47a5d585b41edf627f6c77181d5f04
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ba/78/fb/e0c24a9e73d7483b073d15b7e05f43f3fc2ac75eff6899c7aa
Successfully built wget
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
amzn2extra-docker                                        | 3.0 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
amzn2extra-python3.8                                     | 3.0 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB 

'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

### 1. Setup SageMaker

In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [113]:
%%time
import os
import boto3
import sagemaker

from omegaconf import OmegaConf
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial

from time import strftime
from pathlib import Path

role = get_execution_role()

sm_boto_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session()
region_name = sagemaker_session.boto_region_name

CPU times: user 46.6 ms, sys: 0 ns, total: 46.6 ms
Wall time: 85.6 ms


In [114]:
print(sagemaker.__version__)

2.136.0


## Set Up SageMaker Experiment
Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker.

In [47]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

In [48]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

### 2. Download the NeMo source code

SageMaker allows you to pass in your own source code, with an entrypoint script.

Below we download the AWS NeMo `config.yaml` which contains our configuration, and the `speech_to_text_ctc.py` script to run training.

Our folder structure will look like this:

    code/
        speech_to_text_ctc.py
        conf/
            config.yaml
    data/
        an4/

In [49]:
# root_dir = Path('./an4_nemo_sagemaker/')
# code_dir = root_dir / 'code/'
# config_dir = code_dir / 'conf/'
# data_dir = root_dir / 'data/'

# root_dir.mkdir(exist_ok=True)
# code_dir.mkdir(exist_ok=True)
# config_dir.mkdir(exist_ok=True)
# data_dir.mkdir(exist_ok=True)

# config_path = str(config_dir / "config.yaml")
# # wget.download("https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/conformer/conformer_ctc_char.yaml", config_path)
# # wget.download("https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/asr_ctc/speech_to_text_ctc.py", str(code_dir))

### 2.1 Initialize SageMaker within Training Script

We provide a helper function that we require to be imported and run at the top of the training script.

This installs and setups DDP for you. It also alleviates having to import a custom container, and can leverage all of the SageMaker containers. Rather than running this cell, you could also manually do this in your script.

### 3. Setup the AN4 Dataset, upload data to S3

We now download our training and validation data, uploading to S3 so that SageMaker can mount our data to the instance at runtime.

In [51]:
bucket = sagemaker_session.default_bucket()
prefix = 'nemo-test'
code_location = f's3://{bucket}/{prefix}/backup_codes'
output_path = f's3://{bucket}/{prefix}/model_output' 
s3_log_path = f's3://{bucket}/{prefix}/logs'

In [62]:
data_dir = os.getcwd() + "/dataset/"
data_dir

'/home/ec2-user/SageMaker/2023/training-code/Nvidia-NeMo/nemo-on-sagemaker/NeMo-test/dataset/'

In [66]:
if not os.path.exists(data_dir):
    os.makedirs(data_dir, exist_ok=True)
    
    from nemo.utils.notebook_utils import download_an4
    download_an4(
        data_dir=str(data_dir),
        train_mount_dir="/opt/ml/input/data/training/",
        test_mount_dir="/opt/ml/input/data/testing/",
    )

    # Upload to the default bucket
    prefix = "an4"
    loc = sagemaker_session.upload_data(path=str(data_dir), bucket=bucket, key_prefix=prefix)
else:
    loc = f"s3://{bucket}/{prefix}"
loc

******
Dataset downloaded at: /home/ec2-user/SageMaker/2023/training-code/Nvidia-NeMo/nemo-on-sagemaker/NeMo-test/dataset//an4_sphere.tar.gz
Converting .sph to .wav...
Finished conversion.
******
******
Training manifest created.
Test manifest created.
***Done***


's3://sagemaker-us-west-2-322537213286/an4'

### 4. Configure the training job

Now we configure the training job, by modifying the `config.yaml` file that is stored in our source code directory.
We pass relative directory paths for the data based on the SageMaker mount directory on the remote instance.

In [103]:
config_path = "an4_nemo_sagemaker/code/conf/config.yaml"
conf = OmegaConf.load(config_path)

# Set Data Locations based on the mounted directory in the SageMaker instance
conf.model.train_ds.manifest_filepath = "/opt/ml/input/data/training/an4/train_manifest.json"
conf.model.validation_ds.manifest_filepath = "/opt/ml/input/data/testing/an4/test_manifest.json"
conf.trainer.accelerator = "gpu"
conf.trainer.max_epochs = 5 # 150

# Output directory for our experiment within the SageMaker instance
conf.exp_manager.exp_dir="/opt/ml/model/"

# Create a Small Variant of the Conformer Model
conf.model.encoder.n_layers = 8
conf.model.n_heads = 4
conf.model.spec_augment.time_masks = 5

# Set Optimizer parameters
conf.model.optim.lr = 2.0 # by default we using Noam scheduling, the LR is a multiplier 

OmegaConf.save(conf, config_path)

In [104]:
metric_definitions=[
     {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
     {'Name': 'train_samples_per_second', 'Regex': "train_samples_per_second.*=\D*(.*?)$"},
     {'Name': 'epoch', 'Regex': "epoch.*=\D*(.*?)$"},
     {'Name': 'f1', 'Regex': "f1.*=\D*(.*?)$"},
     {'Name': 'exact_match', 'Regex': "exact_match.*=\D*(.*?)$"}]

In [115]:
experiment_name = 'nemo-exp1'

training_instance_type = 'ml.p3.16xlarge'
# training_instance_type = 'local_gpu'
training_instance_count = 1

do_spot_training = False
max_wait = None
max_run = 1*60*60

In [116]:
kwargs = {}
checkpoint_s3_uri = None

if training_instance_type =='local_gpu':
    from sagemaker.local import LocalSession
    import os
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
    data_channels = {"training": f"file://{data_dir}", "testing": f"file://{data_dir}"}
    
else:
    sagemaker_session = sagemaker.Session()
    data_channels = {"training": loc, "testing": loc}

In [117]:
print("experiment_name : {} \ntrain_instance_type : {} \ntrain_instance_count : {}".format(experiment_name, training_instance_type, instance_count))    

experiment_name : nemo-exp1 
train_instance_type : ml.p3.16xlarge 
train_instance_count : 1


### 5. Run training on SageMaker

Pass the path of the training and validation data on S3 + the output directory on S3 to the PyTorch estimator, and call fit with the appropriate bucket locations for the training and testing data.

In [118]:
image_uri='322537213286.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:nemo-22-12'

In [124]:
role = get_execution_role()

est = PyTorch(
    entry_point="speech_to_text_ctc.py", # the script we want to run
    source_dir=os.getcwd() + '/an4_nemo_sagemaker/code', # where our conf/script is
    role=role,
    instance_type=training_instance_type,
    instance_count=training_instance_count,
    image_uri=image_uri,
    # framework_version="1.13.1", # version of PyTorch
    # py_version="py39",
    volume_size=1024,
    code_location = code_location,
    output_path=output_path,
    disable_profiler=True,
    debugger_hook_config=False,
    hyperparameters={'config-path': 'conf'},
    sagemaker_session=sagemaker_session,
    checkpoint_s3_uri=checkpoint_s3_uri,
    metric_definitions=metric_definitions,
    max_run=max_run,
    **kwargs
)

In [125]:
data_channels

{'training': 's3://sagemaker-us-west-2-322537213286/an4',
 'testing': 's3://sagemaker-us-west-2-322537213286/an4'}

In [130]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

if training_instance_type =='local_gpu':
    est.checkpoint_s3_uri = None
else:
    est.checkpoint_s3_uri = f's3://{bucket}/checkpoints/'


est.fit(
    inputs=data_channels, 
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=False,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: nemo-exp1-0310-16201678465207


In [131]:
job_name=est.latest_training_job.name

In [132]:
sagemaker_session = sagemaker.Session()
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

2023-03-10 16:20:08 Starting - Starting the training job......
2023-03-10 16:21:07 Starting - Preparing the instances for training.........
2023-03-10 16:22:10 Downloading - Downloading input data......................................................................................................................................................
2023-03-10 16:47:48 Training - Training image download completed. Training in progress..=============
== PyTorch ==
NVIDIA Release 22.12 (build 49968248)
PyTorch Version 1.14.0a0+410ce96
Container image Copyright (c) 2022, NVIDIA CORPORATION & AFFILIATES. All rights reserved.
Copyright (c) 2014-2022 Facebook Inc.
Copyright (c) 2011-2014 Idiap Research Institute (Ronan Collobert)
Copyright (c) 2012-2014 Deepmind Technologies    (Koray Kavukcuoglu)
Copyright (c) 2011-2012 NEC Laboratories America (Koray Kavukcuoglu)
Copyright (c) 2011-2013 NYU                      (Clement Farabet)
Copyright (c) 2006-2010 NEC Laboratories America (Ronan Collobert,

### 6. Download model, (Optional) Tensorboard Logs

SageMaker stores our models/logs within a tar file after training has finished. These can be obtained from S3 like below.

We also visualize the training logs. We suggest using an external logger (such as W&B) to track training progress during the run.

In [133]:
key = est.model_data.replace("s3://" + bucket + '/', '')

sagemaker_session.boto_session.client("s3", region_name=region_name).download_file(
    Bucket=bucket, Key=key, Filename='model.tar.gz',
)

In [134]:
!tar -xvzf model.tar.gz

Conformer-CTC-Char1/
Conformer-CTC-Char1/2023-03-10_16-48-03/
Conformer-CTC-Char1/2023-03-10_16-48-03/hparams.yaml
Conformer-CTC-Char1/2023-03-10_16-48-03/nemo_log_globalrank-1_localrank-1.txt
Conformer-CTC-Char1/2023-03-10_16-48-03/nemo_log_globalrank-7_localrank-7.txt
Conformer-CTC-Char1/2023-03-10_16-48-03/nemo_log_globalrank-5_localrank-5.txt
Conformer-CTC-Char1/2023-03-10_16-48-03/nemo_error_log.txt
Conformer-CTC-Char1/2023-03-10_16-48-03/nemo_log_globalrank-2_localrank-2.txt
Conformer-CTC-Char1/2023-03-10_16-48-03/checkpoints/
Conformer-CTC-Char1/2023-03-10_16-48-03/checkpoints/Conformer-CTC-Char1--val_wer=1.0000-epoch=2.ckpt
Conformer-CTC-Char1/2023-03-10_16-48-03/checkpoints/Conformer-CTC-Char1--val_wer=1.0000-epoch=0.ckpt
Conformer-CTC-Char1/2023-03-10_16-48-03/checkpoints/Conformer-CTC-Char1.nemo
Conformer-CTC-Char1/2023-03-10_16-48-03/checkpoints/Conformer-CTC-Char1--val_wer=1.0000-epoch=3.ckpt
Conformer-CTC-Char1/2023-03-10_16-48-03/checkpoints/Conformer-CTC-Char1--val_wer=

In [135]:
%tensorboard --logdir ./ --host 0.0.0.0